# Cortex Compliance AI - Fine-Tuning

Dataset already uploaded to HuggingFace. Just run this notebook on Colab with T4 GPU.

## Steps:
1. Runtime → Change runtime type → **T4 GPU**
2. Run all cells

In [ ]:
# Step 1: Install Unsloth (2x faster, handles all dependencies)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Step 3: Load dataset from HuggingFace (already uploaded!)
from datasets import load_dataset

dataset = load_dataset("maaninder/cortex-compliance-data", split="train")
print(f"✅ Loaded {len(dataset)} training examples from HuggingFace")
print(dataset[0]["text"][:200] + "...")

In [ ]:
# Step 4: Load model with Unsloth (handles 4-bit quantization automatically)
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",  # Pre-quantized!
    max_seq_length=2048,
    load_in_4bit=True,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
)
print("✅ Model loaded with LoRA")

In [ ]:
# Step 5: Train with SFTTrainer
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        output_dir="./cortex-compliance-ai",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        save_strategy="epoch",
        report_to="none",
    ),
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Step 6: Save to HuggingFace Hub
from huggingface_hub import notebook_login
notebook_login()  # Enter your token when prompted

model.push_to_hub("maaninder/cortex-compliance-ai")
tokenizer.push_to_hub("maaninder/cortex-compliance-ai")
print("✅ Model saved to: https://huggingface.co/maaninder/cortex-compliance-ai")

In [ ]:
# Step 7: Test the model
FastLanguageModel.for_inference(model)

test_prompt = "### Instruction:\nGenerate a Personal Data Processing Policy for ООО Тест (INN: 1234567890)\n\n### Response:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Done!

Model: https://huggingface.co/maaninder/cortex-compliance-ai

**Using Unsloth benefits:**
- 2x faster training
- No triton/bitsandbytes errors
- Pre-quantized models
- Works reliably on Colab T4